In [7]:
!python -m pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 3.7 MB/s eta 0:00:00a 0:00:01
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00


In [13]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service
import re
from random import randint
from time import sleep

def run_medrxiv_biorxiv_assistant(search_term):
    my_service = Service(executable_path='/usr/local/bin/chromedriver')
    browser = Browser('chrome', service=my_service)#, headless=True)
    all_dois = []

    print('searching for ' + search_term)

    try:
        # visit bioRxiv's search page
        browser.visit('https://www.biorxiv.org/search')

        # fill in the search form
        sleep(randint(5,10)*0.1)
        print(' fill search term')
        browser.find_by_id('edit-txtsimple').fill(search_term)

        sleep(randint(5,10)*0.1)
        
        print(' click search button')        
        browser.find_by_css('a[class="search-choice-close"]').click()    
        #browser.find_by_css('input[class="form-submit"]').click()    
        browser.find_by_id('edit-actions').find_by_value('Search').click()
        browser.select('numresults', '75')
        
        sleep(randint(5,10)*0.1)
        print(' load next page')
        # Extract the number of results
        formatted_string = browser.find_by_id('page-title').text
        
        # Use regular expressions to get the number from a string formatted 'XXX Results'
        m = re.search(r'\d+ Results', formatted_string)
        if m:
            num_results = int(re.search(r'\d+', formatted_string).group())
            
            loop_count = 0
            while True:
                
                # Extract each result from the list of the web page
                doi_links = browser.find_by_css('span[class="highwire-cite-metadata-doi highwire-cite-metadata"]')
                all_dois.extend([re.sub('doi: ', '', t.text) for t in doi_links])
                
                # Is next button absent?
                next_button_not_present = browser.is_element_not_present_by_css('a[class="link-icon link-icon-after"]')

                if next_button_not_present:
                    break
            
                # Find the next button on the page
                next_button = browser.find_by_css('a[class="link-icon link-icon-after"]')

                # Click the next button
                next_button.click()
                sleep(randint(5,10)*0.1)
                #print('load page number'+str(loop_count))

                loop_count += 1
                if loop_count > 100:
                    break
            
    except Exception as e:
        print(e)

    browser.quit()
    print('found ' + str(len(all_dois)) + ' dois')
    return all_dois


In [18]:
import pandas as pd
from io import StringIO

tsv_text = '''
0	Cellxgene	Cellxgene
1	Cellxgene-vip	cellxgene AND vip
2	Cellxgene-gateway	cellxgene AND gateway
3	CellxGene Discover 	CELLxGENE AND Discover
4	cellxgene-annotate	cellxgene AND annotate
5	StarFish	starfish and transcriptom and zuckerberg
6	CZ Infectious Disease	IDseq OR CZID OR IDSeq OR ID-seq  
7	FastGenomics	fastgenomics
8	HCA DCP	DCP AND Atlas AND HCA
9	CZ Gen EPI	COVID AND Tracker AND zuckerber
10	Napari	napari AND viewer'''

# Create a dataframe from the tsv text
df = pd.read_csv(StringIO(tsv_text), sep='\t', header=None, names=['product_name', 'search_term'])

# create an empty pandas dataframe to store the results
df_dois = pd.DataFrame(columns=['product_name', 'search_term', 'doi'])

# run the assistant for each search term
for index, row in df.iterrows():

    #replace '|' with ' OR ' and '&' with ' AND ' in search term
    search_term = re.sub(r'\|', ' OR ', row['search_term'])
    search_term = re.sub(r'&', ' AND ', search_term)

    dois = run_medrxiv_biorxiv_assistant(row['search_term'])
    df_dois = pd.concat([df_dois, pd.DataFrame({'product_name': row['product_name'], 'search_term': row['search_term'], 'doi': dois})])


searching for Cellxgene
 fill search term
 click search button
 load next page
found 119 dois
searching for cellxgene AND vip
 fill search term
 click search button
 load next page
found 13 dois
searching for cellxgene AND gateway
 fill search term
 click search button
 load next page
found 7 dois
searching for CELLxGENE AND Discover
 fill search term
 click search button
 load next page
found 17 dois
searching for cellxgene AND annotate
 fill search term
 click search button
 load next page
found 108 dois
searching for starfish and transcriptom and zuckerberg
 fill search term
 click search button
 load next page
found 12 dois
searching for IDseq OR CZID OR IDSeq OR ID-seq  
 fill search term
 click search button
 load next page
found 128 dois
searching for fastgenomics
 fill search term
 click search button
 load next page
found 21 dois
searching for DCP AND Atlas AND HCA
 fill search term
 click search button
 load next page
found 9 dois
searching for COVID AND Tracker AND zuckerber

In [19]:
df_dois.to_csv('dois2.csv', sep='\t', index=False)

In [83]:
cellxgene_dois = run_medrxiv_biorxiv_assistant('cellxgene')
print(all_dois)
print(len(all_dois))

118


In [33]:
# Extract each result from the list of the web page

links = browser.find_by_css('span[class="highwire-cite-metadata-doi highwire-cite-metadata"]') 
[t.text for t in links]

['doi: https://doi.org/10.1101/2020.08.28.270652',
 'doi: https://doi.org/10.1101/2021.04.05.438318',
 'doi: https://doi.org/10.1101/2021.09.30.462602',
 'doi: https://doi.org/10.1101/2020.04.17.20069930',
 'doi: https://doi.org/10.1101/2023.01.21.514916',
 'doi: https://doi.org/10.1101/2023.04.30.538439',
 'doi: https://doi.org/10.1101/2021.08.12.455997',
 'doi: https://doi.org/10.1101/2023.02.03.526939',
 'doi: https://doi.org/10.1101/2023.05.17.540842',
 'doi: https://doi.org/10.1101/2022.10.26.513573']

In [ ]:
from random import randint
from time import sleep

# Sleep a random number of seconds (between 1 and 5)
sleep(randint(1,5))

In [45]:
my_service = Service(executable_path='/usr/local/bin/chromedriver')
browser = Browser('chrome', service=my_service, headless=True)

# visit bioRxiv's search page
browser.visit('https://www.biorxiv.org/search')

print(browser.title)

# fill in the search form
browser.find_by_name('txtsimple').fill('cellxgene')

# Sleep a random number of seconds (between 1 and 5)
#sleep(randint(1,5))
#next_button_not_present = browser.is_element_not_present_by_css('a[class="link-icon link-icon-after"]')
#browser.find_by_css('a[class="search-choice-close"]').click()

browser.find_by_id('edit-actions').find_by_value('Search').click()


Advanced Search | bioRxiv


ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" id="search_rightsidebar_submit_1578430535" name="op" value="Search" class="form-submit"> is not clickable at point (416, 583). Other element would receive the click: <p>...</p>
  (Session info: headless chrome=114.0.5735.106)
Stacktrace:
0   chromedriver                        0x00000001097b16b8 chromedriver + 4937400
1   chromedriver                        0x00000001097a8b73 chromedriver + 4901747
2   chromedriver                        0x0000000109366616 chromedriver + 435734
3   chromedriver                        0x00000001093b1b97 chromedriver + 744343
4   chromedriver                        0x00000001093af450 chromedriver + 734288
5   chromedriver                        0x00000001093aca84 chromedriver + 723588
6   chromedriver                        0x00000001093abbe4 chromedriver + 719844
7   chromedriver                        0x000000010939e1e1 chromedriver + 664033
8   chromedri